## Amazon Reviews Sentiment Analysis

# You can test the MODEL here...


In [25]:
test_set = [input("enter the review ")]
new_test = vct.transform(test_set)
clf.predict(new_test)

enter the review Thanks for watching


array(['POSITIVE'], dtype='<U8')

### Building model from here and all  observations are at the end of the file

In [2]:
import random
class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"
    
class Reviews():
    def __init__(self,text, score):
        self.score = score
        self.text = text
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score<= 2:
            return Sentiment.NEGATIVE
        elif self.score ==3:
            return Sentiment.NEUTRAL
        else:   # score of 4 or 5
            return Sentiment.POSITIVE
        
class review_container:
    def __init__(self,reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative = list(filter(lambda x:x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x:x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        #print(negative[0].text)
        print(len(negative))
        print(len(positive_shrunk))

In [3]:
import json    # important to read a json file and work on it..

file_name = "C:\\Users\\P V S Karthik\\Datasets\\Books_small_10000.json"
reviews = []
with open(file_name) as f:   
    for line in f:
        review = json.loads(line)    # loading the json file into a variable...
        reviews.append(Reviews(review["reviewText"],review["overall"])) # creating a list.. of requires objs

In [4]:
reviews[5].text # object Review as created above and used here....

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [5]:
reviews[5].sentiment

'POSITIVE'

From now we are going to use BAG OF WORDS

### Preparation DATA

In [6]:
from sklearn.model_selection import train_test_split as tts
train,test = tts(reviews,test_size = 0.33,random_state = 0)

In [7]:
cont = review_container(train)
cont.evenly_distribute()
train_container = review_container(train)
test_container = review_container(test)
len(cont.reviews)

418
418


836

In [8]:
train_container.evenly_distribute()

train_x = train_container.get_text()
train_y = train_container.get_sentiment()
test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

train_y.count(Sentiment.POSITIVE)
train_y.count(Sentiment.NEGATIVE)

418
418
226
226


418

In [9]:
train_x[0]

"An eBook was provided via NetGalley for an honest review. YA with a bit of the paranormal, this book is suitable for anyone 12+ and above.Every year, I get a flu shot. Every year. So far, it has not affected me adversely. In fact, it has helped cut down the number of days and alleviate the intensity of the flu symptoms when I do get it.Twenty two teenagers from the Bloomberg High School thought the same thing.  Nothing would happen, right? They were wrong. They were prepared for possible headaches, sore arms, possible slight fever. They were not prepared to hear everyone - telepathically. So now they know everyone's secret, what everyone was thinking. That would be perfect, right?Don't Even Think About It is funny, sarcastic, revealing, poignant and engrossing. Be prepared to get immersed in teenage insecurities, angst, crushes, young love and betrayal.Forced to band together to deal with their new sensitivity, personalities who would normally not band together suddenly find themselve

### Bag of words...

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vct = TfidfVectorizer()
train_x_vectors = vct.fit_transform(train_x)
test_x_vectors = vct.transform(test_x)
train_x_vectors
test_x_vectors

<452x8825 sparse matrix of type '<class 'numpy.float64'>'
	with 26027 stored elements in Compressed Sparse Row format>

### Classification

### Support Vector Machine

In [11]:
from sklearn.svm import SVC

clf = SVC(kernel = "linear")
clf.fit(train_x_vectors,train_y)

print(clf.predict(test_x_vectors[0]))

test_x_vectors[0]

['NEGATIVE']


<1x8825 sparse matrix of type '<class 'numpy.float64'>'
	with 58 stored elements in Compressed Sparse Row format>

### Decision Tree

In [12]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(train_x_vectors,train_y)
dt.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Naive Bayes

In [13]:
from sklearn.naive_bayes import GaussianNB
NB = DecisionTreeClassifier()
NB.fit(train_x_vectors,train_y)
NB.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression

lr = DecisionTreeClassifier()
lr.fit(train_x_vectors,train_y)
lr.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

Because our observations are confusing a bit we will do and accuracy test to find which s working well...

### Evaluation

In [15]:
# Mean Accuracy
print(clf.score(test_x_vectors,test_y))
print(dt.score(test_x_vectors,test_y))
print(NB.score(test_x_vectors,test_y))
print(lr.score(test_x_vectors,test_y))

0.8495575221238938
0.6504424778761062
0.6792035398230089
0.668141592920354


Since we got 85% accuracy we are choosingthe SVM model over others...

after changing the Bag of words technique from count vectorizer to the Tfidf vectorizer i have achieved good accuracy.

In [16]:
#f1 score
from sklearn.metrics import f1_score
f1_score(test_y,clf.predict(test_x_vectors),average = None,labels = [Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGATIVE])


C:\Users\P V S Karthik\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


array([0.85152838, 0.        , 0.84753363])

If we can observe the above observations we can say that the positive reviews are more in the dataset so the sataset would be biased

positive - 89%, Neutral = 21%, Negative =7% 

so the dataset is extremely biased or the neutral section is effecting the data..

Based on this now we need to decide which kind of result accuracy is importsnt...

Now i have took more data sample but still this is enough...

But now positive - 90%, Neutral = 26%, Negative =39% 

We have finally built a class tha will equate the number of positive and negative in the training and testing data. So that this will regulate the Bias our model has in the past....

Now the new results are extremely satisfying..... as 

But now positive - 81% Neutral = 26%, Negative = 80 % 

Now we are using the Tfidfvectorizer model instead of count vectorizer as i have just experimented with the cout vectorizer but it is not accurate when compare with the Tfidf .

Count vectorizer actually count the number of positive and negative words in a sentence and based on which it will decide. but when it comes to Tfidf it will reduce the weightage of the repititive  words such as "This","was","is".... etc.... so the accuracy of the model will also increase...

But now positive - 85%, Negative = 84 % 

### Now we have to Tune the model

In [17]:
from sklearn.model_selection import GridSearchCV
parameters = {"kernel":("linear","rbf"),"C":(1,4,8,16,32)}
svc = SVC()
clf = GridSearchCV(svc,parameters,cv = 5)
clf.fit(train_x_vectors,train_y)



GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [18]:
print(clf.score(test_x_vectors,test_y))

0.838495575221239


We can maek this model better by cleanign the data even further just like this example may effect the prediction...

Ex: "good! VS good" there is a high chance of these two will be segregated as different words...

## Saving the Model

In [19]:
import pickle

Save the model

In [20]:
with open('C:\\Users\\P V S Karthik\\projects\\sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

Load the model

In [21]:
with open('C:\\Users\\P V S Karthik\\projects\\sentiment_classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

Testing the model using pickle

In [22]:

print(test_x[0])

loaded_clf.predict(test_x_vectors[0])

I gave this book a shot. Look, having kids is the best thing one can do, in my opinion. I just want to make sure my kids are productive members of society. I don't have the time to read about complaining about kids. I enjoy them, it's hard work but what you sign up for!


array(['POSITIVE'], dtype='<U8')